In [2]:
from sklearn import datasets, metrics, linear_model
from sklearn.model_selection import train_test_split
import os
import numpy as np 
import pandas as pd

In [3]:
dir_data = '../data/ml100marathon/'
D49_train = os.path.join(dir_data, 'train_data.csv')
D49_test_features = os.path.join(dir_data, 'test_features.csv')
# D49_sample_submission = os.path.join(dir_data, 'sample_submission.csv')


train = pd.read_csv(D49_train)
# sample = pd.read_csv(D49_sample_submission)

In [4]:
train.dtypes.value_counts()

float64    19
object      2
bool        1
dtype: int64

In [5]:
train.head()

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0


In [6]:
train["from_this_person_to_poi"].describe()

count     65.000000
mean      40.092308
std       88.901407
min        0.000000
25%        0.000000
50%        7.000000
75%       27.000000
max      411.000000
Name: from_this_person_to_poi, dtype: float64

In [7]:
train["from_poi_to_this_person"].describe()

count     65.000000
mean      64.800000
std       91.863214
min        0.000000
25%       10.000000
50%       28.000000
75%       88.000000
max      528.000000
Name: from_poi_to_this_person, dtype: float64

In [8]:
train[train["poi"] == True]["from_poi_to_this_person"].describe()

count      9.000000
mean      93.888889
std       78.588238
min       13.000000
25%       42.000000
50%       53.000000
75%      123.000000
max      240.000000
Name: from_poi_to_this_person, dtype: float64

In [9]:
train[train["poi"] == True]["from_this_person_to_poi"].describe()

count     9.000000
mean     18.777778
std      13.718398
min       4.000000
25%      11.000000
50%      15.000000
75%      25.000000
max      48.000000
Name: from_this_person_to_poi, dtype: float64

In [14]:
train[train["poi"] == True]

,name,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
0,RICE KENNETH D,1750000.0,NaN,-3504386.0,NaN,ken.rice@enron.com,19794175.0,46950.0,18.0,42.0,...,1617011.0,174839.0,True,2748364.0,NaN,420636.0,864.0,905.0,505050.0,22542539.0
1,SKILLING JEFFREY K,5600000.0,NaN,NaN,NaN,jeff.skilling@enron.com,19250000.0,29336.0,108.0,88.0,...,1920000.0,22122.0,True,6843672.0,NaN,1111258.0,2042.0,3627.0,8682716.0,26093672.0
2,SHELBY REX,200000.0,NaN,-4167.0,NaN,rex.shelby@enron.com,1624396.0,22884.0,39.0,13.0,...,NaN,1573324.0,True,869220.0,NaN,211844.0,91.0,225.0,2003885.0,2493616.0
3,KOPPER MICHAEL J,800000.0,NaN,NaN,NaN,michael.kopper@enron.com,NaN,118134.0,NaN,NaN,...,602671.0,907502.0,True,985032.0,NaN,224305.0,NaN,NaN,2652612.0,985032.0
4,CALGER CHRISTOPHER F,1250000.0,NaN,-262500.0,NaN,christopher.calger@enron.com,NaN,35818.0,144.0,199.0,...,375304.0,486.0,True,126027.0,NaN,240189.0,2188.0,2598.0,1639297.0,126027.0
5,HIRKO JOSEPH,NaN,10259.0,NaN,NaN,joe.hirko@enron.com,30766064.0,77978.0,NaN,NaN,...,NaN,2856.0,True,NaN,NaN,NaN,NaN,NaN,91093.0,30766064.0
6,YEAGER F SCOTT,NaN,NaN,NaN,NaN,scott.yeager@enron.com,8308552.0,53947.0,NaN,NaN,...,NaN,147950.0,True,3576206.0,NaN,158403.0,NaN,NaN,360300.0,11884758.0
7,COLWELL WESLEY,1200000.0,27610.0,-144062.0,NaN,wes.colwell@enron.com,NaN,16514.0,40.0,240.0,...,NaN,101740.0,True,698242.0,NaN,288542.0,1132.0,1758.0,1490344.0,698242.0
8,LAY KENNETH L,7000000.0,202911.0,-300000.0,NaN,kenneth.lay@enron.com,34348384.0,99832.0,36.0,123.0,...,3600000.0,10359729.0,True,14761694.0,NaN,1072321.0,2411.0,4273.0,103559793.0,49110078.0
9,GLISAN JR BEN F,600000.0,NaN,NaN,NaN,ben.glisan@enron.com,384728.0,125978.0,16.0,52.0,...,71023.0,200308.0,True,393818.0,NaN,274975.0,874.0,873.0,1272284.0,778546.0


In [10]:
print(train["from_this_person_to_poi"])
print(train["from_poi_to_this_person"])

0        4.0
1       30.0
2       14.0
3        NaN
4       25.0
5        NaN
6        NaN
7       11.0
8       16.0
9        6.0
10      15.0
11      48.0
12       NaN
13       NaN
14      65.0
15       2.0
16       NaN
17       NaN
18       NaN
19      14.0
20       0.0
21       NaN
22       0.0
23       1.0
24       0.0
25       NaN
26     194.0
27       0.0
28       3.0
29     171.0
       ...  
83       0.0
84       NaN
85       NaN
86       5.0
87     387.0
88       NaN
89       8.0
90       NaN
91       0.0
92       NaN
93       0.0
94       NaN
95       0.0
96       4.0
97       6.0
98       NaN
99       NaN
100      NaN
101      2.0
102      2.0
103      0.0
104      NaN
105      NaN
106     19.0
107      NaN
108      NaN
109      1.0
110      0.0
111      7.0
112     65.0
Name: from_this_person_to_poi, Length: 113, dtype: float64
0       42.0
1       88.0
2       13.0
3        NaN
4      199.0
5        NaN
6        NaN
7      240.0
8      123.0
9       52.0
10      53.0
11   

In [15]:
tmp = train.pop("name")
tmp = train.pop("email_address")

In [16]:
train.dtypes.value_counts()

float64    19
bool        1
dtype: int64

In [17]:
train["from_this_person_to_poi"].replace(np.NaN, 0, inplace=True)
train["from_poi_to_this_person"].replace(np.NaN, 0, inplace=True)

In [18]:
print(train.columns)
print(train.select_dtypes(include=["float"]).columns)
# print(np.median(train[~train["bonus"].isnull()]["bonus"]))
for column in train.select_dtypes(include=["float"]).columns:
    print(column)
    train[column].replace(np.NaN, np.median(train[~train[column].isnull()][column]), inplace=True)


Index(['bonus', 'deferral_payments', 'deferred_income', 'director_fees',
       'exercised_stock_options', 'expenses', 'from_messages',
       'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
       'long_term_incentive', 'other', 'poi', 'restricted_stock',
       'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
       'to_messages', 'total_payments', 'total_stock_value'],
      dtype='object')
Index(['bonus', 'deferral_payments', 'deferred_income', 'director_fees',
       'exercised_stock_options', 'expenses', 'from_messages',
       'from_poi_to_this_person', 'from_this_person_to_poi', 'loan_advances',
       'long_term_incentive', 'other', 'restricted_stock',
       'restricted_stock_deferred', 'salary', 'shared_receipt_with_poi',
       'to_messages', 'total_payments', 'total_stock_value'],
      dtype='object')
bonus
deferral_payments
deferred_income
director_fees
exercised_stock_options
expenses
from_messages
from_poi_to_this_person
from_thi

In [19]:
for column in train.columns:
    print(column, train[column].isnull().sum())
#     break

bonus 0
deferral_payments 0
deferred_income 0
director_fees 0
exercised_stock_options 0
expenses 0
from_messages 0
from_poi_to_this_person 0
from_this_person_to_poi 0
loan_advances 0
long_term_incentive 0
other 0
poi 0
restricted_stock 0
restricted_stock_deferred 0
salary 0
shared_receipt_with_poi 0
to_messages 0
total_payments 0
total_stock_value 0


In [20]:
target = train.pop("poi")
x_train, x_test, y_train, y_test = train_test_split(train, target, test_size=0.25, random_state=4)

In [21]:
liner_reg = linear_model.LinearRegression()
liner_reg.fit(x_train, y_train)
y_pred = liner_reg.predict(x_test)
auc = metrics.roc_auc_score(y_test, y_pred)
print("AUC: ", auc)
print(y_pred)

AUC:  0.7400000000000001
[-0.01540027 -0.04912744  0.05461777 -0.04982838  0.30246438  0.0799838
  0.97242905 -0.03569102  0.05525922  0.08256243 -0.02411958 -0.23414687
  0.00408165  0.40435264  0.09309507  0.08361886  0.25790121  0.35548167
  0.06515072 -0.01951624  0.0289556   0.22166461 -0.03327042 -0.05437323
  0.05141272 -0.02324335 -0.01061796 -0.03781931  0.29736646]


In [22]:
logi_reg = linear_model.LogisticRegression()
logi_reg.fit(x_train, y_train)
y_pred = logi_reg.predict(x_test)
auc = metrics.roc_auc_score(y_test, y_pred)
print("AUC: ", auc)
print(y_pred)

AUC:  0.625
[False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False False]


In [29]:
lasso_reg = linear_model.Lasso(alpha=1.0)
lasso_reg.fit(x_train, y_train)
y_pred = lasso_reg.predict(x_test)
auc = metrics.roc_auc_score(y_test, y_pred)
print("AUC: ", auc)
print(y_pred)

AUC:  0.7600000000000001
[ 0.00725309 -0.02153578  0.06699308 -0.03197031  0.28924388  0.04049295
  0.98469407 -0.02901645  0.06432166  0.1023845  -0.01142313 -0.22662427
  0.01565597  0.33992895  0.09460032  0.08494458  0.26873017  0.34489307
  0.06981957 -0.01553152  0.03574834  0.22302634 -0.01371047 -0.03941729
  0.07524355 -0.00348692  0.14129023 -0.0035207   0.30045889]


/Users/tommy/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [30]:
ridge_reg = linear_model.Ridge(alpha=1.0)
ridge_reg.fit(x_train, y_train)
y_pred = ridge_reg.predict(x_test)
auc = metrics.roc_auc_score(y_test, y_pred)
print("AUC: ", auc)
print(y_pred)

AUC:  0.7400000000000001
[-0.01540022 -0.04912727  0.05461787 -0.04982832  0.30246475  0.07998301
  0.97243008 -0.03569121  0.05525932  0.0825625  -0.02411965 -0.23414623
  0.00408175  0.40435157  0.09309503  0.08361885  0.25790146  0.35548171
  0.06515082 -0.01951636  0.02895569  0.22166471 -0.03327037 -0.05437329
  0.0514129  -0.02324324 -0.01061482 -0.03781899  0.29736704]


In [31]:
test = pd.read_csv(D49_test_features)
name = test.pop("name")
tmp = test.pop("email_address")
for column in test.select_dtypes(include=["float"]).columns:
#     print(column)
    test[column].replace(np.NaN, np.median(test[~test[column].isnull()][column]), inplace=True)
for column in train.columns:
    print(column, train[column].isnull().sum())

bonus 0
deferral_payments 0
deferred_income 0
director_fees 0
exercised_stock_options 0
expenses 0
from_messages 0
from_poi_to_this_person 0
from_this_person_to_poi 0
loan_advances 0
long_term_incentive 0
other 0
restricted_stock 0
restricted_stock_deferred 0
salary 0
shared_receipt_with_poi 0
to_messages 0
total_payments 0
total_stock_value 0


In [32]:
pred = lasso_reg.predict(test)
print(pred)

[ 0.13573392  0.50727065  0.38349992  0.10423495  0.54391038  0.44606364
  0.37033627  0.16430947  0.20344332  0.10099783 -0.08623168  0.11646969
  0.07156287  0.05392145  0.37415174  0.40620481  0.11922267  0.44619752
  0.62328592  0.06558114 24.66902044  0.0996252   0.56709791  0.20959346
  0.45215864  0.14830008  4.43471095  0.15679591  0.02575032  0.16565002
  0.12019751  0.14491903  0.22623883]


In [33]:
data = {
    "name" : name
}
data["poi"] = pred
output = pd.DataFrame(data)
output.head()

,name,poi
0,BELDEN TIMOTHY N,0.135734
1,BOWEN JR RAYMOND M,0.507271
2,HANNON KEVIN P,0.383500
3,DELAINEY DAVID W,0.104235
4,CAUSEY RICHARD A,0.543910


In [34]:
value = output["poi"].values
output["poi"] = ( value - min(value) ) / ( max(value) - min(value) )
output["poi"]

0     0.008966
1     0.023975
2     0.018975
3     0.007694
4     0.025455
5     0.021502
6     0.018443
7     0.010121
8     0.011702
9     0.007563
10    0.000000
11    0.008188
12    0.006374
13    0.005662
14    0.018597
15    0.019892
16    0.008299
17    0.021508
18    0.028661
19    0.006133
20    1.000000
21    0.007508
22    0.026392
23    0.011950
24    0.021749
25    0.009474
26    0.182626
27    0.009817
28    0.004524
29    0.010175
30    0.008339
31    0.009337
32    0.012622
Name: poi, dtype: float64

In [35]:
output.to_csv("../data/d49_submit_01.csv", index=False)